# Generating Multiple Sequence Alignement given PDB structure

Given a PDB structure file we use ProDy to search all available Pfam alignments for the best matching Multiple Sequence Alignment (MSA)

In [1]:
# Initial Import

# System packages
import os.path, sys
from pathlib import Path


# Scientific Computing packages
import numpy as np
np.random.seed(1) # set random seed for reproducibility
import pandas as pd
from scipy import linalg
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder
from scipy.spatial import distance_matrix, distance


# Biopython packages
import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
from Bio import SeqIO, pairwise2

# Parallelization and computing diagnosti packages
from joblib import Parallel, delayed
import timeit

# Plotting packages
import matplotlib.pyplot as plt

# # --- Import our Code ---# #
# Direct Coupling Analysis code
from direct_info import direct_info

# Import data processing and general DCA_ER tools
from data_processing import data_processing_pdb2msa, pdb2msa
import tools
# # -----------------------# #

# Import ProDy code.
from prody import *

# Define Computational Resources
n_cpus = 10

## Define Data directories
* data_path: Where is the MSA and pdb structure data
* out_dir: Where do you want the processed MSA data to go
* pdb_dir: Where do you want the PDB-MSA reference and structure data to go

In [2]:

# Define data directories -- NEED TO DEFINE
data_path = Path('example_protein_data')
DCA_ER_dir = '.' # Set DCA_ER directory
out_dir = '%s/protein_data/di/' % DCA_ER_dir
processed_data_dir = "%s/protein_data/data_processing_output" % DCA_ER_dir
pdb_dir = '%s/protein_data/pdb_data/' % DCA_ER_dir


if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    os.makedirs(processed_data_dir)
    os.makedirs(pdb_dir)

    
# Path to gzipped pdb file
pdb_path = "/pdb/pdb/zd/pdb1zdr.ent.gz"


## Un-zip and load PDB file to get PDB structure (1zdr)

In [3]:
import gzip, shutil
def gunzip(file_path, output_path):
    print('Unzipping %s to %s' % (file_path, output_path))
    with gzip.open(file_path,"rb") as f_in, open(output_path,"wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

        
        
unzipped_pdb_filename = os.path.basename(pdb_path).replace(".gz", "")
pdb_out_path = "%s%s" % (pdb_dir, unzipped_pdb_filename)
gunzip(pdb_path, pdb_out_path)

Unzipping /pdb/pdb/zd/pdb1zdr.ent.gz to ./protein_data/pdb_data/pdb1zdr.ent


## Generate PDB2MSA Data
* For each available polypeptide sequence in the PDB structure us ProDy to BLAST for corresponding sequences in the Pfam domain

In [4]:
pdb2msa_results = pdb2msa(pdb_out_path, pdb_dir, create_new=False)
print('PDB2MSA Results:\n', pdb2msa_results)

if len(pdb2msa_results) > 1:
    fasta_file = pdb2msa_results[0]
    prody_df = pdb2msa_results[1]
else:
    prody_df = pdb2msa_results[0]


print('\n\n\nPDB DF with associated Protein Families\n', prody_df.loc[:,  [column for column in prody_df.columns if column not in ['locations', 'PDB Sequence']]].head())

PDB2MSA Results:
 [   Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1 PDB ID Chain  Polypeptide Index  \
0           0             0               0   1zdr     B                  0   
1           1             1               1   1zdr     A                  0   

      Pfam   accession   class      id  ... ali_end ali_start  bitscore  end  \
0  PF00186  PF00186.22  Domain  DHFR_1  ...     160         1    215.11  160   
1  PF00186  PF00186.22  Domain  DHFR_1  ...     160         1    215.08  160   

    cond_evalue    ind_evalue    evidence  hmm_end hmm_start  start  
0  2.300000e-68  4.500000e-64  hmmer v3.0      160         1      1  
1  2.300000e-68  4.600000e-64  hmmer v3.0      160         1      1  

[2 rows x 22 columns]]



PDB DF with associated Protein Families
    Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1 PDB ID Chain  Polypeptide Index  \
0           0             0               0   1zdr     B                  0   
1           1             1               1   1zdr     A          

## Load and Pre-Process Pfam-MSA
* With the PDB2MSA data we can now loop through all the different PDB-MSA pairings for all available sequences
* Because the PDB2MSA data is ordered by PDB-polypeptide sequence to MSA alignment score the we will loop from most similar to least similar as we load and Pre-Process the different MSAs for our analysis

In [5]:
print("\n\n\nLooping through Prody Search DataFrame:\n", prody_df.head())
rows_to_drop = []
for ir, pdb2msa_row in enumerate(prody_df.iterrows()):
    print('\n\nGetting msa with following pdb2msa entry:\n', pdb2msa_row)
    #try:
    dp_result =  data_processing_pdb2msa(data_path, prody_df.iloc[pdb2msa_row[0]], gap_seqs=0.2, gap_cols=0.2, prob_low=0.004,
                               conserved_cols=0.8, printing=True, out_dir=processed_data_dir, pdb_dir=pdb_dir, letter_format=False,
                               remove_cols=True, create_new=True)
    if dp_result is not None:
        [s0, removed_cols, s_index, tpdb, pdb_s_index, bad_seq_indx] = dp_result
        break
    else:
        rows_to_drop.append(ir)
        continue



pdb_id = pdb2msa_row[1]['PDB ID']
pfam_id = pdb2msa_row[1]['Pfam']
chain = pdb2msa_row[1]['Chain']
# update Prody search DF (use same filename as pdb2msa() in data_processing
prody_df = prody_df.drop(rows_to_drop)
print("\n\n\nSaving updated Prody Search DataFrame:\n", prody_df.head())
prody_df.to_csv('%s/%s_pdb_df.csv' % (pdb_dir, pdb_id))

if dp_result is None:
    print('None of the available prody pdb search found matching alignments... Exiting..')
    sys.exit()
print('Done Preprocessing Data.....')



# number of positions
n_var = s0.shape[1]
n_seq = s0.shape[0]

print("Number of residue positions:",n_var)
print("Number of sequences:",n_seq)

# Compute effective number of sequences
dst = distance.squareform(distance.pdist(s0, 'hamming'))
theta = .2 # minimum necessary distance (theta = 1. - seq_identity_thresh)
seq_ints = (dst < theta).sum(axis=1).astype(float)
ma_inv = 1/((dst < theta).sum(axis=1).astype(float))  
meff = ma_inv.sum()

print('N_effective ', meff)





Looping through Prody Search DataFrame:
    Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1 PDB ID Chain  Polypeptide Index  \
0           0             0               0   1zdr     B                  0   
1           1             1               1   1zdr     A                  0   

      Pfam   accession   class      id  ... ali_end ali_start  bitscore  end  \
0  PF00186  PF00186.22  Domain  DHFR_1  ...     160         1    215.11  160   
1  PF00186  PF00186.22  Domain  DHFR_1  ...     160         1    215.08  160   

    cond_evalue    ind_evalue    evidence  hmm_end hmm_start  start  
0  2.300000e-68  4.500000e-64  hmmer v3.0      160         1      1  
1  2.300000e-68  4.600000e-64  hmmer v3.0      160         1      1  

[2 rows x 22 columns]


Getting msa with following pdb2msa entry:
 (0, Unnamed: 0                                                           0
Unnamed: 0.1                                                         0
Unnamed: 0.1.1                                       

MISHIVAMDENRVIGKDNRLPWHLPADLAYFKRVTMGHAIVMGRKTFEAIGRPLPGRDNVVVTG-NRSFRPEGCLVLHSLEEVKQWIASRADEVFIIGGAELFRATMPIVDRLYVTKIFASFPGDTFYPPISDDEWEIVSYTPGGKDEKNPYEHAFIIYER
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
MISHIVAMDENRVIGKDNRLPWHLPADLAYFKRVTMGHAIVMGRKTFEAIGRPLPGRDNVVVT-RNRSFRPEGCLVLHSLEEVKQWIASRADEVFIIGGAELFRATMPIVDRLYVTKIFASFPGDTFYPPISDDEWEIVSYTPGGKDEKNPYEHAFIIYER
  Score=159

match upgrade at 69
69: pairwise score=159.000000

lengths:  160 160
Alignment(seqA='MISHIVAMDENRVIGKDNRLPWHLPADLAYFKRVTMGHAIVMGRKTFEAIGRPLPGRDNVVVTG-NRSFRPEGCLVLHSLEEVKQWIASRADEVFIIGGAELFRATMPIVDRLYVTKIFASFPGDTFYPPISDDEWEIVSYTPGGKDEKNPYEHAFIIYER', seqB='MISHIVAMDENRVIGKDNRLPWHLPADLAYFKRVTMGHAIVMGRKTFEAIGRPLPGRDNVVVT-RNRSFRPEGCLVLHSLEEVKQWIASRADEVFIIGGAELFRATMPIVDRLYVTKIFASFPGDTFYPPISDDEWEIVSYTPGGKDEKNPYEHAFIIYER', score=159.0, start=0, end=161)
best match is sequence 69 with hamming distance of 

[[ 7 15  4 ...  3 18 18]
 [ 9  0  9 ... 17 17 19]
 [20 20 17 ... 20 20 20]
 ...
 [17 15  9 ... 11 17 19]
 [17 15  0 ...  8 17 19]
 [ 4 15  7 ...  7  7 19]]
(6696, 132)
In Data Processing Final Reference Sequence (shape= (132,) ): 
 [ 7 15  6  7 17 10  2  3 11 14 17  8  2 11 14  9 12  6  9 12  0  9  0 19
  8 14 17 10  5  6  0  7 17  8  4  3  7  5 14 12  5  2 11 17 17 16 11 14
 15  4 14 12  3  5  1  9 17  9  6 15  9  3  3 17  8 13 18  7  0 15 14  2
  3 17  4  7  7  0  3  9  4 14  0 16 10 12  7 17  2 14  9 19 17  8  7  4
  0 15  4 12  5 16  4 19 12  7 15  2  2  3 18  3  7 17 15 19 16 12  5  5
  3  8 11 12 19  3  6  0  4  7  7 19]



Saving updated Prody Search DataFrame:
    Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1 PDB ID Chain  Polypeptide Index  \
0           0             0               0   1zdr     B                  0   
1           1             1               1   1zdr     A                  0   

      Pfam   accession   class      id  ... ali_end ali_start  bitscore  end  \
0  P

## Use Expectation Reflection to calculate Mutual Information
* Re-Define MSA using OneHot transformation to get binary variables for ER
* Calculate in parallel

### OneHot tranformation and initialization of weight matix and fields

In [6]:
# --- Get number of binary aa-pair-position variables ie aa-aa pair at bosition i (yes/no) ---- #
# number of aminoacids at each position #
mx = np.array([len(np.unique(s0[:,i])) for i in range(n_var)]) #
print("Number of different amino acids at each position:\n",mx) #

mx_cumsum = np.insert(mx.cumsum(),0,0) 
i1i2 = np.stack([mx_cumsum[:-1],mx_cumsum[1:]]).T  
mx_sum = mx.sum() 
print("\nTotal number of variables:",mx_sum) 
 # number of bias term 
n_linear = mx_sum - n_var 
onehot_encoder = OneHotEncoder(sparse=False,categories='auto') 
# s is OneHot encoder format, s0 is original sequnce matrix #
s = onehot_encoder.fit_transform(s0)
# --------------------------------------------------------------------------------------------- #


# Define wight matrix with variable for each possible amino acid at each sequence position
w = np.zeros((mx.sum(),mx.sum())) 
h0 = np.zeros(mx.sum())

Number of different amino acids at each position:
 [ 6 16 12  7  8 16  8 17 12 14  9 19 11 10 13  5  5 14  9  6 11  9 12 11
  9 10 16 17 11 12 13  5  4  9  7 11  5 10 12  9  9 17  8  8  5  2 11 18
 18 20 20 20 14 13 15 16 15 14 14  7 12 15 12  8 15 15 21 19 16 16 20 15
 17 13 11  8 10 12 13  4  3 13 15 12 11 13 19  9 11 17  7 14  8 16  3 16
 19 15 14 15  7  7 15  7 17 10 16 20 17 18  5 17 16 17 20 19 18 21 21 19
 15 17 13 16 13 14 10 14  9 18 14  5]

Total number of variables: 1671


### Define Expectation Reflection Model

In [7]:
# Expectation Reflection
#=========================================================================================
def predict_w(s,i0,i1i2,niter_max,l2):
    #print('i0:',i0)
    i1,i2 = i1i2[i0,0],i1i2[i0,1]

    x = np.hstack([s[:,:i1],s[:,i2:]])
    y = s[:,i1:i2]

    h01,w1 = ER.fit(x,y,niter_max,l2)

    return h01,w1

In [8]:
import expectation_reflection as ER
# parallel
start_time = timeit.default_timer()
#res = Parallel(n_jobs = 4)(delayed(predict_w)\
#res = Parallel(n_jobs = 8)(delayed(predict_w)\
res = Parallel(n_jobs = n_cpus-2)(delayed(predict_w)\
        (s,i0,i1i2,niter_max=10,l2=100.0)\
        for i0 in range(n_var))

run_time = timeit.default_timer() - start_time
print('run time:',run_time)
for i0 in range(n_var):
    i1,i2 = i1i2[i0,0],i1i2[i0,1]

    h01 = res[i0][0]
    w1 = res[i0][1]

    h0[i1:i2] = h01
    w[:i1,i1:i2] = w1[:i1,:]
    w[i2:,i1:i2] = w1[i1:,:]

# make w symmetric
w = (w + w.T)/2.

# Verify that w is symmetric (sanity test)
print("Dimensions of w: ",w.shape)

di = direct_info(s0,w)
print(di)
print(di.shape)

run time: 240.41063459217548
Dimensions of w:  (1671, 1671)
[[0.         0.11951145 0.01811685 ... 0.00571031 0.00500088 0.00641744]
 [0.11951145 0.         0.02567006 ... 0.00519084 0.01089716 0.01033743]
 [0.01811685 0.02567006 0.         ... 0.0056172  0.00570035 0.00810997]
 ...
 [0.00571031 0.00519084 0.0056172  ... 0.         0.03656981 0.10436078]
 [0.00500088 0.01089716 0.00570035 ... 0.03656981 0.         0.092877  ]
 [0.00641744 0.01033743 0.00810997 ... 0.10436078 0.092877   0.        ]]
(132, 132)


In [9]:
# Verify that w is symmetric (sanity test)


np.save("%s/%s_%s_ER_di.npy" % (out_dir, pdb_id, pfam_id), di)

# Convert DI matrix to DI score dictionary (pydca format)
DI_ER_pydca = tools.scores_matrix2dict(di, s_index)
for site_pair, score in DI_ER_pydca[:5]:
    print(site_pair, score)
import pickle as pkl
with open("%s/%s_%s_ER_di_pydca.npy" % (out_dir, pdb_id, pfam_id), "wb") as f_pydca:
    pkl.dump(DI_ER_pydca,f_pydca )
f_pydca.close()


(12, 121) 0.29758608228091077
(49, 50) 0.19325603923675688
(35, 55) 0.16890810058887018
(57, 72) 0.16461237928399758
(39, 58) 0.1494621805552581
